In [1]:
%matplotlib inline

import torch
import torch.nn.functional as F
from torch.utils import data
from torch.autograd import Variable
from torchvision import models
import h5py
import json

from preprocessing import *
from crowdnet_pytorch import CrowdNet

/home/samuel_cahyawijaya/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
model_name = 'cc_epoch_15.mdl'
model_path = os.path.expanduser(os.path.join('./models/demo_dcc_crowdnet', model_name))
dataset_path = 'dataset/UCF_CC_50/'
shanghai_train_path = 'dataset/ShanghaiTech/Train/'
shanghai_test_path = 'dataset/ShanghaiTech/Test/'

In [3]:
images, gts, gts_count, densities = load_images_and_gts(dataset_path)

dataset/UCF_CC_50/37.json
dataset/UCF_CC_50/20.json
dataset/UCF_CC_50/20.json 62.50520805512529 637.2357512953367 (<class 'IndexError'>, IndexError('index 637 is out of bounds for axis 1 with size 600',), <traceback object at 0x7f596e2a9f88>)
dataset/UCF_CC_50/36.json
dataset/UCF_CC_50/8.json
dataset/UCF_CC_50/3.json
dataset/UCF_CC_50/15.json
dataset/UCF_CC_50/43.json
dataset/UCF_CC_50/19.json
dataset/UCF_CC_50/41.json
dataset/UCF_CC_50/26.json
dataset/UCF_CC_50/47.json
dataset/UCF_CC_50/34.json
dataset/UCF_CC_50/11.json
dataset/UCF_CC_50/44.json
dataset/UCF_CC_50/1.json
dataset/UCF_CC_50/1.json 149.34820247110426 984.2566389281416 (<class 'IndexError'>, IndexError('index 984 is out of bounds for axis 1 with size 984',), <traceback object at 0x7f596e2ab708>)
dataset/UCF_CC_50/1.json 533.5036428856122 984.2566389281419 (<class 'IndexError'>, IndexError('index 984 is out of bounds for axis 1 with size 984',), <traceback object at 0x7f596e2ab708>)
dataset/UCF_CC_50/5.json
dataset/UCF_CC_5

In [4]:
model = CrowdNet()
model.load_state_dict(torch.load(model_path))
# if torch.cuda.is_available():
#     model.cuda()

In [5]:
print(images[0].reshape(1, images[0].shape[2], images[0].shape[0], -1).shape)

(1, 3, 768, 1024)


In [6]:
predicted_count = []
for i in range(len(images)):
    input_img = Variable(torch.Tensor(images[i].reshape(1, images[i].shape[2], images[i].shape[0], -1)))
#     if torch.cuda.is_available():
#         input_img.cuda()

    predicted = model(input_img)
    predicted_count.append(predicted.data.sum().numpy())

In [7]:
def load_images_and_gts_shanghai(path):
    images = []
    gts = []
    gts_count = []
    densities = []
    for gt_file in glob.iglob(os.path.join(path, '*.json')):
        print(gt_file)
        if os.path.isfile(gt_file.replace('.json','.png').replace('GT_','')):
            img = cv2.imread(gt_file.replace('.json','.png').replace('GT_',''))
        else:
            img = cv2.imread(gt_file.replace('.json','.jpg').replace('GT_',''))
        images.append(img)
        
        #load ground truth
        gt, count = load_gt_from_json_shanghai(gt_file, img.shape[:-1])
        gts.append(gt)
        gts_count.append(count)
        
        #densities
        desnity_file = gt_file.replace('.json','.h5')
        if os.path.isfile(desnity_file):
            #load density if exist
            with h5py.File(desnity_file, 'r') as hf:
                density = np.array(hf.get('density'))
        else:
            density = gaussian_filter_density([gt])[0]
            with h5py.File(desnity_file, 'w') as hf:
                hf['density'] = density
        densities.append(density)
    print(path, len(images), 'img loaded')
    print(path, len(densities), 'den loaded')
    return (images, gts, gts_count, densities)


def load_gt_from_json_shanghai(gt_file, gt_shape):
    gt = np.zeros(gt_shape, dtype='uint8') 
    with open(gt_file, 'r') as jf:
        json_data = json.load(jf)
        for j, dot in enumerate(json_data):
            try:
                gt[int(math.floor(dot['y'])), int(math.floor(dot['x']))] = 1
            except IndexError:
                print(gt_file, dot['y'], dot['x'], sys.exc_info())
    return gt, len(json_data)

In [8]:
sh_tr_images, sh_tr_gts, sh_tr_gts_count, sh_tr_densities = load_images_and_gts_shanghai(shanghai_train_path)

dataset/ShanghaiTech/Train/GT_IMG_130.json
dataset/ShanghaiTech/Train/GT_IMG_156.json
dataset/ShanghaiTech/Train/GT_IMG_144.json
dataset/ShanghaiTech/Train/GT_IMG_226.json
dataset/ShanghaiTech/Train/GT_IMG_28.json
dataset/ShanghaiTech/Train/GT_IMG_92.json
dataset/ShanghaiTech/Train/GT_IMG_118.json
dataset/ShanghaiTech/Train/GT_IMG_268.json
dataset/ShanghaiTech/Train/GT_IMG_154.json
dataset/ShanghaiTech/Train/GT_IMG_146.json
dataset/ShanghaiTech/Train/GT_IMG_167.json
dataset/ShanghaiTech/Train/GT_IMG_169.json
dataset/ShanghaiTech/Train/GT_IMG_83.json
dataset/ShanghaiTech/Train/GT_IMG_173.json
dataset/ShanghaiTech/Train/GT_IMG_26.json
dataset/ShanghaiTech/Train/GT_IMG_197.json
dataset/ShanghaiTech/Train/GT_IMG_283.json
dataset/ShanghaiTech/Train/GT_IMG_153.json
dataset/ShanghaiTech/Train/GT_IMG_215.json
dataset/ShanghaiTech/Train/GT_IMG_174.json
dataset/ShanghaiTech/Train/ 20 img loaded
dataset/ShanghaiTech/Train/ 20 den loaded


In [9]:
predicted_count_sh_tr = []
for i in range(len(sh_tr_images)):
    input_img = Variable(torch.Tensor(sh_tr_images[i].reshape(1, sh_tr_images[i].shape[2], sh_tr_images[i].shape[0], -1)))
#     if torch.cuda.is_available():
#         input_img.cuda()

    predicted = model(input_img)
    predicted_count_sh_tr.append(predicted.data.sum().numpy())

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 1. Got 75 and 76 in dimension 2 at /opt/conda/conda-bld/pytorch_1524586445097/work/aten/src/TH/generic/THTensorMath.c:3586

In [11]:
sh_ts_images, sh_ts_gts, sh_ts_gts_count, sh_ts_densities = load_images_and_gts_shanghai(shanghai_test_path)

dataset/ShanghaiTech/Test/GT_IMG_104.json
dataset/ShanghaiTech/Test/GT_IMG_130.json
dataset/ShanghaiTech/Test/GT_IMG_96.json
dataset/ShanghaiTech/Test/GT_IMG_144.json
dataset/ShanghaiTech/Test/GT_IMG_90.json
dataset/ShanghaiTech/Test/GT_IMG_181.json
dataset/ShanghaiTech/Test/GT_IMG_125.json
dataset/ShanghaiTech/Test/GT_IMG_5.json
dataset/ShanghaiTech/Test/GT_IMG_92.json
dataset/ShanghaiTech/Test/GT_IMG_75.json
dataset/ShanghaiTech/Test/GT_IMG_134.json
dataset/ShanghaiTech/Test/GT_IMG_86.json
dataset/ShanghaiTech/Test/GT_IMG_131.json
dataset/ShanghaiTech/Test/GT_IMG_17.json
dataset/ShanghaiTech/Test/GT_IMG_11.json
dataset/ShanghaiTech/Test/GT_IMG_169.json
dataset/ShanghaiTech/Test/GT_IMG_115.json
dataset/ShanghaiTech/Test/GT_IMG_83.json
dataset/ShanghaiTech/Test/GT_IMG_161.json
dataset/ShanghaiTech/Test/GT_IMG_15.json
dataset/ShanghaiTech/Test/GT_IMG_66.json
dataset/ShanghaiTech/Test/GT_IMG_150.json
dataset/ShanghaiTech/Test/GT_IMG_20.json
dataset/ShanghaiTech/Test/GT_IMG_172.json
datas

In [12]:
predicted_count_sh_ts = []
for i in range(len(sh_ts_images)):
    input_img = Variable(torch.Tensor(sh_ts_images[i].reshape(1, sh_ts_images[i].shape[2], sh_ts_images[i].shape[0], -1)))
#     if torch.cuda.is_available():
#         input_img.cuda()

    predicted = model(input_img)
    predicted_count_sh_ts.append(predicted.data.sum().numpy())

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 1. Got 91 and 92 in dimension 3 at /opt/conda/conda-bld/pytorch_1524586445097/work/aten/src/TH/generic/THTensorMath.c:3586

# Evaluation

In [10]:
for i in range(len(predicted_count)):
    print('{}-th'.format(i),predicted_count[i], gts_count[i], predicted_count[i] - gts_count[i])
print('Total', sum(predicted_count), sum(gts_count), sum(predicted_count) - sum(gts_count), (sum(predicted_count) - sum(gts_count)) / len(predicted_count))

0-th 7356.0737 917 6439.07373046875
1-th 2460.852 2706 -245.14794921875
2-th 17070.873 2105 14965.873046875
3-th 19059.918 1046 18013.91796875
4-th 9336.03 2364 6972.0302734375
5-th 5103.6646 1543 3560.66455078125
6-th 6335.653 947 5388.65283203125
7-th 1237.1349 754 483.1348876953125
8-th 4477.3296 1284 3193.32958984375
9-th 8969.056 2391 6578.0556640625
10-th 3851.7375 1961 1890.737548828125
11-th 10311.002 1115 9196.001953125
12-th 8346.365 484 7862.365234375
13-th 7596.233 2358 5238.23291015625
14-th 12555.729 4633 7922.728515625
15-th 3917.1382 469 3448.13818359375
Total 127984.78894042969 27077 100907.78894042969 6306.7368087768555


In [13]:
for i in range(len(predicted_count_sh_tr)):
    print('{}-th'.format(i),predicted_count_sh_tr[i], sh_tr_gts_count[i], predicted_count_sh_tr[i] - sh_tr_gts_count[i])
print('Total', sum(predicted_count_sh_tr), sum(sh_tr_gts_count), sum(predicted_count_sh_tr) - sum(sh_tr_gts_count), (sum(predicted_count_sh_tr) - sum(sh_tr_gts_count)) / len(predicted_count_sh_tr))

0-th 5411.016 1177 4234.01611328125
1-th 11655.305 728 10927.3046875
2-th 24120.615 777 23343.615234375
Total 41186.93603515625 18004 23182.93603515625 7727.645345052083


In [14]:
for i in range(len(predicted_count_sh_ts)):
    print('{}-th'.format(i),predicted_count_sh_ts[i], sh_ts_gts_count[i], predicted_count_sh_ts[i] - sh_ts_gts_count[i])
print('Total', sum(predicted_count_sh_ts), sum(sh_ts_gts_count), sum(predicted_count_sh_ts) - sum(sh_ts_gts_count), (sum(predicted_count_sh_ts) - sum(sh_ts_gts_count)) / len(predicted_count_sh_ts))

0-th 8669.66 1175 7494.66015625
1-th 5932.0894 921 5011.08935546875
2-th 2010.1533 246 1764.1533203125
3-th 8395.468 262 8133.4677734375
4-th 6391.384 2256 4135.3837890625
5-th 16875.645 216 16659.64453125
6-th 13768.848 269 13499.84765625
7-th 18455.596 817 17638.595703125
8-th 17689.375 1366 16323.375
9-th 4760.85 665 4095.85009765625
10-th 9588.267 165 9423.2666015625
11-th 13349.479 579 12770.478515625
12-th 18411.209 604 17807.208984375
13-th 6135.9077 1156 4979.90771484375
14-th 11841.161 1068 10773.1611328125
15-th 2714.4692 218 2496.46923828125
16-th 1443.6036 1191 252.6036376953125
17-th 23863.281 218 23645.28125
18-th 1100.4116 266 834.41162109375
19-th 5102.5713 341 4761.5712890625
Total 196499.42736816406 18009 178490.42736816406 8924.521368408203
